In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import dgl
import dgl.function as fn
import pandas as pd
import json

with open('./ERM_E/info.dat', 'r') as load_f:
    info_dict = json.load(load_f)
    link_info = info_dict['link.dat']['link type']
    node_info = info_dict['node.dat']['node type']

print(link_info)
# read nodes
nodes = pd.read_csv('./ERM_E/node.dat', sep="\t", header=None)
edges = pd.read_csv('./ERM_E/link.dat', sep="\t", header=None)
label_train = pd.read_csv('./ERM_E/label.dat', sep="\t", header=None)
label_test = pd.read_csv('./ERM_E/label.dat.test', sep="\t", header=None)

{'0': {'start': 0, 'end': 0, 'meaning': 'TFGI'}, '1': {'start': 2, 'end': 0, 'meaning': 'CPI'}, '2': {'start': 0, 'end': 1, 'meaning': 'GRI'}, '3': {'start': 2, 'end': 1, 'meaning': 'MRI'}, '4': {'start': 1, 'end': 2, 'meaning': 'RMI'}, '5': {'start': 0, 'end': 0, 'meaning': 'SFGI'}, '6': {'start': 0, 'end': 0, 'meaning': 'PPI'}, '7': {'start': 1, 'end': 3, 'meaning': 'RPI'}}


In [2]:
# add node features
paper_feature = []
author_feature = []
subject_feature = []

for i in range(len(nodes)):
    if (nodes.iloc[i, 2] == 0):
        paper_feature.append(list(map(float, nodes.iloc[i, 3].split(','))))
    else:
        continue

meta_graphs = {}
for i in range(8):
    edge = edges[edges[2] == i]
    source_node = edge.iloc[:, 0].values - np.min(edge.iloc[:, 0].values)  # 使link.dat中的每种节点id从0开始
    target_node = edge.iloc[:, 1].values - np.min(edge.iloc[:, 1].values)
    meta_graphs[(node_info[str(link_info[str(i)]['start'])], link_info[str(i)]['meaning'], node_info[str(link_info[str(i)]['end'])])] = (torch.tensor(source_node), torch.tensor(target_node))
g = dgl.heterograph(meta_graphs)
g

Graph(num_nodes={'Gene': 3035, 'Metabolite': 1059, 'Pathway': 38, 'Reaction': 2373},
      num_edges={('Gene', 'GRI', 'Reaction'): 4297, ('Gene', 'PPI', 'Gene'): 9059, ('Gene', 'SFGI', 'Gene'): 2211, ('Gene', 'TFGI', 'Gene'): 4552, ('Metabolite', 'CPI', 'Gene'): 3644, ('Metabolite', 'MRI', 'Reaction'): 2627, ('Reaction', 'RMI', 'Metabolite'): 2710, ('Reaction', 'RPI', 'Pathway'): 2375},
      metagraph=[('Gene', 'Reaction', 'GRI'), ('Gene', 'Gene', 'PPI'), ('Gene', 'Gene', 'SFGI'), ('Gene', 'Gene', 'TFGI'), ('Reaction', 'Metabolite', 'RMI'), ('Reaction', 'Pathway', 'RPI'), ('Metabolite', 'Gene', 'CPI'), ('Metabolite', 'Reaction', 'MRI')])

In [3]:
meta_graphs

{('Gene', 'TFGI', 'Gene'): (tensor([   0,    0,    0,  ..., 3010, 3010, 3010]),
  tensor([   3,    4,  640,  ..., 3022, 3023, 3024])),
 ('Metabolite',
  'CPI',
  'Gene'): (tensor([   0,    0,    0,  ..., 1055, 1055, 1055]), tensor([  70,  153,  188,  ..., 1609, 1910, 2711])),
 ('Gene',
  'GRI',
  'Reaction'): (tensor([   0,    3,    3,  ..., 3026, 3026, 3029]), tensor([ 615,  399, 1964,  ...,  252,  563,  245])),
 ('Metabolite',
  'MRI',
  'Reaction'): (tensor([   0,    0,    1,  ..., 1056, 1056, 1057]), tensor([1311, 2262,  219,  ..., 1721, 1776, 1566])),
 ('Reaction',
  'RMI',
  'Metabolite'): (tensor([   0,    0,    0,  ..., 2369, 2370, 2371]), tensor([ 12,  28, 444,  ..., 876, 380, 590])),
 ('Gene', 'SFGI', 'Gene'): (tensor([   0,    0,    0,  ..., 1011, 1011, 1011]),
  tensor([   8,   64,  131,  ..., 3008, 3009, 3010])),
 ('Gene', 'PPI', 'Gene'): (tensor([   0,    0,    0,  ..., 3029, 3029, 3029]),
  tensor([  45,  282, 2247,  ..., 2200, 2837, 2839])),
 ('Reaction',
  'RPI',
  'Pa

In [4]:
sad

NameError: name 'sad' is not defined

In [ ]:
# assign node featuer
embedding_dim = 1280
node_embedding = nn.Embedding(3, embedding_dim)


g.nodes['Gene'].data['h'] = torch.FloatTensor(paper_feature)
g.nodes['Metabolite'].data['h'] = node_embedding(torch.tensor([0] * g.num_nodes('Metabolite')))
g.nodes['Pathway'].data['h'] = node_embedding(torch.tensor([1] * g.num_nodes('Pathway')))
g.nodes['Reaction'].data['h'] = node_embedding(torch.tensor([2] * g.num_nodes('Reaction')))

# get idx for train
train_paper_id = torch.LongTensor(label_train.iloc[:, 0].values)
# get idx for test
test_paper_id = torch.LongTensor(label_test.iloc[:, 0].values)

# get labels in graph for train and test
train_paper_label_value = torch.FloatTensor(label_train.iloc[:, 3].values)
test_paper_label_value = torch.FloatTensor(label_test.iloc[:, 3].values)

train_paper_mask_value = torch.full((1, len(train_paper_id)), True).bool()
test_paper_mask_value = torch.full((1, len(test_paper_id)), True).bool()

train_paper_mask = torch.full((len(paper_feature), ), False).bool()
test_paper_mask = torch.full((len(paper_feature), ), False).bool()

paper_label = torch.full((len(paper_feature),), -1.)

paper_label[train_paper_id] = train_paper_label_value
paper_label[test_paper_id] = test_paper_label_value

train_paper_mask[train_paper_id] = train_paper_mask_value
test_paper_mask[test_paper_id] = test_paper_mask_value

g.nodes['Gene'].data['label'] = paper_label
g.nodes['Gene'].data['train_mask'] = train_paper_mask
g.nodes['Gene'].data['test_mask'] = test_paper_mask
# save graph
dgl.save_graphs("./erm.bin", g)

In [ ]:
g

Graph(num_nodes={'Gene': 3035, 'Metabolite': 1059, 'Pathway': 38, 'Reaction': 2373},
      num_edges={('Gene', 'GRI', 'Reaction'): 4297, ('Gene', 'PPI', 'Gene'): 9102, ('Gene', 'SFGI', 'Gene'): 2211, ('Gene', 'TFGI', 'Gene'): 4552, ('Metabolite', 'CPI', 'Gene'): 3753, ('Metabolite', 'MRI', 'Reaction'): 2943, ('Reaction', 'RMI', 'Metabolite'): 3026, ('Reaction', 'RPI', 'Pathway'): 2375},
      metagraph=[('Gene', 'Reaction', 'GRI'), ('Gene', 'Gene', 'PPI'), ('Gene', 'Gene', 'SFGI'), ('Gene', 'Gene', 'TFGI'), ('Reaction', 'Metabolite', 'RMI'), ('Reaction', 'Pathway', 'RPI'), ('Metabolite', 'Gene', 'CPI'), ('Metabolite', 'Reaction', 'MRI')])

Graph(num_nodes={'Gene': 3035, 'Metabolite': 1059, 'Pathway': 38, 'Reaction': 2373},
      num_edges={('Gene', 'GRI', 'Reaction'): 4297, ('Gene', 'PPI', 'Gene'): 9102, ('Gene', 'SFGI', 'Gene'): 2211, ('Gene', 'TFGI', 'Gene'): 4552, ('Metabolite', 'CPI', 'Gene'): 3753, ('Metabolite', 'MRI', 'Reaction'): 2943, ('Reaction', 'RMI', 'Metabolite'): 3026, ('Reaction', 'RPI', 'Pathway'): 2375},
      metagraph=[('Gene', 'Reaction', 'GRI'), ('Gene', 'Gene', 'PPI'), ('Gene', 'Gene', 'SFGI'), ('Gene', 'Gene', 'TFGI'), ('Reaction', 'Metabolite', 'RMI'), ('Reaction', 'Pathway', 'RPI'), ('Metabolite', 'Gene', 'CPI'), ('Metabolite', 'Reaction', 'MRI')])
我想提取('Gene', 'Gene', 'TFGI')全部边 然后取出10% 然后从原图中抹掉这些边

In [ ]:
u, v = g.edges(etype='TFGI')
edges = list(zip(u.numpy(), v.numpy()))

# 步骤 2: 选择这些边的 10%
num_edges_to_remove = len(edges) // 10
indices_to_remove = np.random.choice(len(edges), num_edges_to_remove, replace=False)
edges_to_remove = {('Gene', 'Gene', 'TFGI'): [(edges[i][0], edges[i][1]) for i in indices_to_remove]}

# 步骤 3: 从图中删除这些边
# 在 DGL 中，您不能直接从图中删除边。您需要创建一个新图，不包含那些边。
u, v = g.edges(form='uv', order='eid', etype='TFGI')
u, v = u.numpy(), v.numpy()
edges_to_keep = {('Gene', 'Gene', 'TFGI'): [(u[i], v[i]) for i in range(len(u)) if i not in indices_to_remove]}

In [ ]:
# 创建新的子图，不包含被移除的边
subg = dgl.edge_subgraph(g, edges_to_keep)

# subg 是更新后的图